# Library

In [1]:
import numpy as np
import torch
import gpytorch
import argparse
import time
import pickle
import scipy.io as sio

from torch.distributions import Normal
import matplotlib.pyplot as plt
import random

import roslib
import rospy
import tf as tf_ros
from nav_msgs.msg import Odometry, Path
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
from geometry_msgs.msg import PoseStamped, PoseArray, Pose
import math
import cv2
import copy

import sys
sys.path.append('..')

the rosdep view is empty: call 'sudo rosdep init' and 'rosdep update'


## Check GPU

In [2]:
import torch
import numpy as np
import sys
sys.path.append('..')

from torchlib.utils import list_device,set_device

# S1: check GPU
#list_device()

# S2: default parameters
set_device(1)
np.set_printoptions(precision = 2)
torch.set_default_dtype(torch.float32)
torch.set_printoptions(precision=4)
torch.backends.cudnn.benchmark = True
torch.set_printoptions(sci_mode=False)

Using Device 1 : TITAN Xp


# Set Arguments

In [3]:
import argparse
import sys
import os
import time
import pickle

parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', type=int, default=420, help='size of mini batch')
parser.add_argument('--is_normalization', type=bool, default=True, help='whether do data normalization')
parser.add_argument('--target_image_size', default=[300, 300], nargs=2, type=int, help='Input images will be resized to this for data argumentation.')
parser.add_argument('--output_dim', default=3, type=int, help='output dimention.')
parser.add_argument('--feat_dim', default=128, type=int, help='feature dimention.')
parser.add_argument('--model_dir', type=str, default='/notebooks/global_localization/gps_net_torch', help='rnn, gru, or lstm')

parser.add_argument('--test_dataset', type=str, default=[# '/notebooks/michigan_nn_data/2012_01_08',
                                                         # '/notebooks/michigan_nn_data/2012_01_15',
                                                         # '/notebooks/michigan_nn_data/2012_01_22',
                                                         # '/notebooks/michigan_nn_data/2012_02_02',
                                                         # '/notebooks/michigan_nn_data/2012_02_04',
                                                         # '/notebooks/michigan_nn_data/2012_02_05',
                                                         '/notebooks/michigan_nn_data/2012_02_12',
                                                         # '/notebooks/michigan_nn_data/2012_03_31',
                                                         '/notebooks/michigan_nn_data/2012_04_29',
                                                         '/notebooks/michigan_nn_data/2012_05_11',
                                                         '/notebooks/michigan_nn_data/2012_06_15',
                                                         '/notebooks/michigan_nn_data/2012_08_04',
                                                         # '/notebooks/michigan_nn_data/2012_09_28'])
                                                         '/notebooks/michigan_nn_data/2012_10_28',
                                                         '/notebooks/michigan_nn_data/2012_11_16',
                                                         '/notebooks/michigan_nn_data/2012_12_01'
                                                        ] )

parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/test'])
parser.add_argument('--norm_tensor', type=str, default = ['/notebooks/global_localization/norm_mean_std.pt'])

#parser.add_argument('--map_dataset', type=str, default='/home/kevin/data/michigan_gt/training')
parser.add_argument('--enable_ros', type=bool, default=False, help='put data into ros')
parser.add_argument('--cuda_device', type=int, default=1, help='cuda device')

sys.argv = ['']
args = parser.parse_args()

if args.enable_ros:
    rospy.init_node('global_localization_tf_broadcaster_cnn')

# Load Dataset

In [4]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import tf.transformations as tf_tran
from tqdm import tqdm
#from PIL import Image
import numpy as np
import random

import torch.nn as nn
import torch.optim as optim
from torchlib import resnet, vggnet, cnn_auxiliary
from torchlib.cnn_auxiliary import normalize, denormalize, denormalize_navie, get_relative_pose, translational_rotational_loss
from torchlib.utils import LocalizationDataset, display_loss, data2tensorboard
import time

transform = transforms.Compose([transforms.ToTensor()])
dataset = LocalizationDataset(dataset_dirs = args.test_dataset, \
                              image_size = args.target_image_size, \
                              transform = transform,
                              get_pair = False, mode='evaluate', sampling_rate=1)

[args.norm_mean, args.norm_std] = torch.load(*args.norm_tensor)
print('Load norm and std:',*args.norm_tensor)

dataloader = DataLoader(dataset, batch_size=args.batch_size, \
                        shuffle=False, num_workers=0, \
                        drop_last=False, pin_memory=True)

100%|██████████| 25367/25367 [00:50<00:00, 498.06it/s]


Load norm and std: /notebooks/global_localization/norm_mean_std.pt


# Define Model

In [5]:
from torch.cuda.amp import autocast, GradScaler
from torchlib.GPs import Backbone, NN, GPNode, BaseModule
    
class Model(nn.Module):
    def __init__(self, num_gp = 20, sub_feat_rate = 0.6666, feat_dim = 128, output_dim = 3):
        super().__init__()
        self.backbone = Backbone()
        self.nn = NN()
        self.gps = nn.ModuleList()
        
        self.num_gp = num_gp
        self.sub_feat_rate = sub_feat_rate
        self.sub_feat_dim = int(feat_dim*self.sub_feat_rate)
        
        for i in range(self.num_gp):
            inducing_points = torch.zeros(output_dim, 300, self.sub_feat_dim)
            # use i as seed to fix sub features
            gp = GPNode(inducing_points,seed=i)
            self.gps.append(gp)
        
    def forward_nn(self, input_data):
        dense_feat = self.backbone(input_data)
        output, feature_t, feature_r = self.nn(dense_feat)
        rot_pred = torch.split(output, [3, 4], dim=1)[1] # 4-dimention            
        return feature_t, rot_pred
    
    def forward_gp(self,gp,trans_feat):
        sub_trans_feat = trans_feat[:,gp.feat_index]
        trans_pred = gp(sub_trans_feat)
        return trans_pred
    
class PosePredictor(BaseModule):
    def __init__(self, norm_mean, norm_std, args,
                 regressor_context_rate = [0.0,0.0],
                 is_training=True, train_rot = True):
        
        super().__init__(norm_mean, norm_std, args)
        self.model = Model().to(self.device)
        self.train_rot = train_rot
        
        # disable learning backbone
        self.disable_requires_grad(self.model.backbone)
        
        if is_training:
            # training tool
            self.optimizer = optim.Adam(self._optimize(regressor_context_rate))
            self.scheduler = optim.lr_scheduler.LambdaLR(optimizer=self.optimizer,
                                                             lr_lambda=lambda epoch: args.decay_rate**epoch)
        else:
            self.disable_requires_grad(self.model)
            
    def _optimize(self, regressor_context_rate = [0.0,0.0]):
        optimizer = [
                {'params': self.model.gps.parameters(), \
                 'lr': self.args.learning_rate,'weight_decay':self.args.weight_decay}]
            
        # NN
        if regressor_context_rate[0]!=0:
            optimizer += [{'params': self.model.nn.global_regressor.parameters(), \
                 'lr': args.learning_rate * regressor_context_rate[0],'weight_decay':args.weight_decay}]
            print('Regressor learn rate:',regressor_context_rate[0])
        else:
            self.disable_requires_grad(self.model.nn.global_regressor)
                
        if regressor_context_rate[1]!=0:
            optimizer += [{'params': self.model.nn.global_context.parameters(), \
                 'lr': args.learning_rate * regressor_context_rate[1],'weight_decay':args.weight_decay}]
            print('Context learn rate:',regressor_context_rate[1])
            self.train_rot = True
        else:
            self.disable_requires_grad(self.model.nn.global_context)
            
        
        if not self.train_rot and regressor_context_rate[1]==0.0:
            self.disable_requires_grad(self.model.nn.global_regressor.regressor.fc1_rot)
            self.disable_requires_grad(self.model.nn.global_regressor.regressor.fc2_rot)
            self.disable_requires_grad(self.model.nn.global_regressor.regressor.fc3_rot)
            self.disable_requires_grad(self.model.nn.global_regressor.regressor.logits_r)
                
        return optimizer
    
    def train(self, x, y):
        # Step 0: zero grad
        self.optimizer.zero_grad()
        
        start = time.time()
        # Step 1: get data
        x,y = x.to(self.device),y.to(self.device)
        if self.args.is_normalization:
            y = normalize(y,self.norm_mean, self.norm_std)
            
        # Step 2: training
        assert self.model.training == True
        
        trans_loss = torch.tensor(0.).to(self.device)
        
        trans_target, rot_target = torch.split(y, [3, 4], dim=1)
        trans_feat, rot_pred = self.model.forward_nn(x)
        rot_loss = self._nn_loss(rot_pred,rot_target)
        for i,gp in enumerate(self.model.gps):
            #sampled_mask = torch.randint(high=args.batch_size, size=(self.model.sub_batch_size,))
            sampled_mask = torch.randint(high=self.args.batch_size, size=(self.args.batch_size,))
            sub_x = trans_feat[sampled_mask]
            sub_y = trans_target[sampled_mask]
            gp_loss = self._gp_loss(gp,sub_x,sub_y)
            trans_loss += gp_loss
        trans_loss = trans_loss/self.model.num_gp
        
        total_loss = trans_loss + self.args.lamda_weights * rot_loss
        
        batch_time = time.time() - start
        
        #Step 3: update
        total_loss.backward()
        self.optimizer.step()
        
        return float(total_loss), batch_time    
    
    def _nn_loss(self, rot_pred, rot_target):
        rot_loss = 1. - torch.mean(torch.square(torch.sum(torch.mul(rot_pred,rot_target),dim=1)))
        return rot_loss
        
    def _gp_loss(self, gp, trans_feat, trans_target):
        # predict
        trans_pred = self.model.forward_gp(gp,trans_feat)
        
        #num_data = int(min(len(dataloader)*args.batch_size,len(dataset))*self.model.sub_batch_rate)
        num_data = min(len(dataloader)*self.args.batch_size,len(dataset))
        mll = gpytorch.mlls.PredictiveLogLikelihood(gp.likelihood, gp.gp, num_data = num_data)
        
        # trans loss
        trans_loss = -1.*mll(trans_pred, trans_target)
        
        return trans_loss
    
    def _eval_gp(self, gp, trans_pred):
        c_mean, c_var = trans_pred.mean, trans_pred.variance
        y_mean, y_var = gp.likelihood(trans_pred).mean, gp.likelihood(trans_pred).variance
        
        return y_mean, c_mean, c_var
    
    def _sample(self, mean, var, num_sample = 100):
        dist = torch.distributions.Normal(mean, var)
        samples = dist.sample([num_sample])
        return samples

    def eval_forward(self, x, y, num_sample = 100, output_denormalize = True):
        # Step 1: get data
        x = x.to(self.device)
        
        # Step 2: forward
        assert self.model.training == False
        trans_feat, rot_pred = self.model.forward_nn(x)
        
        trans_preds = 0
        trans_means = 0
        trans_vars = 0
        for gp in self.model.gps:
            trans_pred = self.model.forward_gp(gp,trans_feat)
            trans_pred, trans_mean, trans_var = self._eval_gp(gp, trans_pred)
            trans_preds += trans_pred * 1/trans_var
            trans_means += trans_mean * 1/trans_var
            trans_vars += 1/trans_var
         
        trans_vars = 1/trans_vars
        trans_preds *= trans_vars
        trans_means *= trans_vars
        
        if self.args.is_normalization and output_denormalize:
            trans_preds = denormalize_navie(trans_preds, self.norm_mean, self.norm_std)
            trans_means = denormalize_navie(trans_means, self.norm_mean, self.norm_std)
            trans_vars = trans_vars.mul(self.norm_std)
        
        samples = self._sample(trans_means, trans_vars, num_sample)
        
        # Step 3: split output
        trans_target, rot_target = torch.split(y, [3, 4], dim=1)
        
        return trans_preds, rot_pred, trans_target, rot_target, samples

In [6]:
trainer = PosePredictor(args.norm_mean,args.norm_std,args,is_training = False)
trainer.load_model('pretrained_gp20_fix.pth')

Successfully loaded model to TITAN Xp.


# Initialize

In [7]:
trans_errors = []
rot_errors = []
uncertainties = []
pose_map = []

total_trans_error = 0.
total_rot_error = 0.

count = 0.

is_save_map = False
is_read_map = False

trans_preds = []
trans_gts = []

rot_preds = []
rot_gts = []

pred_uncertainties = []

pred_time = []

br = tf_ros.TransformBroadcaster()

GT_POSE_TOPIC = '/gt_pose'
BIRDVIEW_TOPIC_PUB = '/bird_view'
MAP_TOPIC_PUB = '/pose_map'
PARTICLES_PUB = '/particles'
NN_LOCALIZASION_PUB = '/nn_pose'
gt_pose_pub = rospy.Publisher(GT_POSE_TOPIC, Odometry, queue_size=1)
bird_view_pub = rospy.Publisher(BIRDVIEW_TOPIC_PUB, Image, queue_size=1)
map_pub = rospy.Publisher(MAP_TOPIC_PUB, Path, queue_size=1)
particles_pub = rospy.Publisher(PARTICLES_PUB, PoseArray, queue_size=1)
nn_pose_pub = rospy.Publisher(NN_LOCALIZASION_PUB, Odometry, queue_size=1)

In [8]:
trainer.model.eval()

for b, data in enumerate(dataloader, 0):
    start = time.time()
    x,y = data.values()
    trans_pred, rot_pred, trans_gt, rot_gt, samples = trainer.eval_forward(x,y)
    
    # transform data
    trans_pred = trans_pred.cpu().numpy()
    rot_pred = rot_pred.cpu().numpy()
    trans_gt = trans_gt.cpu().numpy()
    rot_gt = rot_gt.cpu().numpy()
    
    end = time.time()
    
    if args.enable_ros:
        particles = PoseArray()
        particles.header.stamp = rospy.Time.now()
        particles.header.frame_id = 'world'
        for s in samples:
            pose = Pose()
            [pose.position.x, pose.position.y, pose.position.z] = s
            [pose.orientation.x, pose.orientation.y, pose.orientation.z, pose.orientation.w] = rot_pred[0]
            particles.poses.append(pose)
        particles_pub.publish(particles)

        [px_pred, py_pred, pz_pred] = trans_pred[0]
        [qx_pred, qy_pred, qz_pred, qw_pred] = rot_pred[0]

        br.sendTransform((px_pred, py_pred, pz_pred),
                         (qx_pred, qy_pred, qz_pred, qw_pred), rospy.Time.now(),
                         "estimation", "world")

        [px_gt, py_gt, pz_gt] = trans_gt[0]
        [qx_gt, qy_gt, qz_gt, qw_gt] = rot_gt[0]

        br.sendTransform((px_gt, py_gt, pz_gt),
                         (qx_gt, qy_gt, qz_gt, qw_gt),
                         rospy.Time.now(), "gt", "world")

        timestamp = rospy.Time.now()

        nn_pose_msg = Odometry()
        nn_pose_msg.header.frame_id = 'world'
        nn_pose_msg.header.stamp = timestamp
        nn_pose_msg.child_frame_id = 'base_link'
        nn_pose_msg.pose.pose.position.x = px_pred
        nn_pose_msg.pose.pose.position.y = py_pred
        nn_pose_msg.pose.pose.position.z = pz_pred
        [nn_pose_msg.pose.pose.orientation.x, nn_pose_msg.pose.pose.orientation.y, nn_pose_msg.pose.pose.orientation.z, nn_pose_msg.pose.pose.orientation.w] = [qx_pred, qy_pred, qz_pred, qw_pred]

        conv = np.zeros((6,6), dtype=np.float32)
        [conv[0][0], conv[1][1], conv[2][2]] = trans_cov[0]
        nn_pose_msg.pose.covariance = conv.flatten().tolist()
        nn_pose_pub.publish(nn_pose_msg)

        bridge = CvBridge()

        bird_view_img_msg = bridge.cv2_to_imgmsg(np.asarray(x[0].cpu(), dtype=np.float32), encoding="passthrough")
        stamp_now = rospy.Time.now()
        bird_view_img_msg.header.stamp = stamp_now

        bird_view_pub.publish(bird_view_img_msg)

        rospy.sleep(.0)
        cv2.waitKey(0)

        count += 1
    else:
        count += y.shape[0]
    
    trans_preds += [x for x in trans_pred]
    rot_preds += [x for x in rot_pred]
    trans_gts += [x for x in trans_gt]
    rot_gts += [x for x in rot_gt]

    trans_error = np.sqrt(np.sum((trans_pred - trans_gt)**2,axis=1))
    rot_error_1 = np.arccos(np.sum(np.multiply(rot_pred,rot_gt),axis=1))/math.pi*180
    rot_error_2 = np.arccos(np.sum(np.multiply(rot_pred,-rot_gt),axis=1))/math.pi*180
    rot_error = np.minimum(rot_error_1,rot_error_2)

    trans_errors += [x for x in trans_error]
    rot_errors += [x for x in rot_error]

    total_trans_error += np.sum(trans_error)
    total_rot_error += np.sum(rot_error)
    
    display = 1

    if b % display == 0:
        print(
            "{}/{}, translation error = {:.3f}, rotation error = {:.3f}, time/batch = {:.3f}"
            .format(
             (b+1)*args.batch_size,
            len(dataloader)*args.batch_size,
            total_trans_error / count,
            total_rot_error / count,
            end - start))

420/183960, translation error = 2.574, rotation error = 3.699, time/batch = 5.282
840/183960, translation error = 8.915, rotation error = 4.822, time/batch = 1.278
1260/183960, translation error = 6.956, rotation error = 4.369, time/batch = 1.336
1680/183960, translation error = 6.124, rotation error = 4.273, time/batch = 1.261
2100/183960, translation error = 7.135, rotation error = 4.344, time/batch = 1.259
2520/183960, translation error = 6.644, rotation error = 4.670, time/batch = 1.260
2940/183960, translation error = 6.124, rotation error = 4.705, time/batch = 1.257
3360/183960, translation error = 5.509, rotation error = 4.687, time/batch = 1.257
3780/183960, translation error = 5.107, rotation error = 4.804, time/batch = 1.262
4200/183960, translation error = 4.990, rotation error = 5.170, time/batch = 1.254
4620/183960, translation error = 4.703, rotation error = 5.051, time/batch = 1.258
5040/183960, translation error = 4.458, rotation error = 4.999, time/batch = 1.256
5460/1

41580/183960, translation error = 4.301, rotation error = 4.812, time/batch = 1.273
42000/183960, translation error = 4.270, rotation error = 4.807, time/batch = 1.271
42420/183960, translation error = 4.242, rotation error = 4.836, time/batch = 1.311
42840/183960, translation error = 4.219, rotation error = 4.830, time/batch = 1.270
43260/183960, translation error = 4.280, rotation error = 4.837, time/batch = 1.273
43680/183960, translation error = 5.676, rotation error = 4.966, time/batch = 1.271
44100/183960, translation error = 7.728, rotation error = 5.506, time/batch = 1.271
44520/183960, translation error = 9.607, rotation error = 5.798, time/batch = 1.268
44940/183960, translation error = 9.610, rotation error = 5.769, time/batch = 1.273
45360/183960, translation error = 9.555, rotation error = 5.755, time/batch = 1.267
45780/183960, translation error = 9.482, rotation error = 5.754, time/batch = 1.271
46200/183960, translation error = 9.439, rotation error = 5.764, time/batch 

82740/183960, translation error = 8.582, rotation error = 5.738, time/batch = 1.277
83160/183960, translation error = 8.551, rotation error = 5.727, time/batch = 1.279
83580/183960, translation error = 8.514, rotation error = 5.727, time/batch = 1.264
84000/183960, translation error = 8.482, rotation error = 5.724, time/batch = 1.268
84420/183960, translation error = 8.521, rotation error = 5.735, time/batch = 1.273
84840/183960, translation error = 8.496, rotation error = 5.718, time/batch = 1.271
85260/183960, translation error = 8.470, rotation error = 5.702, time/batch = 1.277
85680/183960, translation error = 8.443, rotation error = 5.689, time/batch = 1.273
86100/183960, translation error = 8.412, rotation error = 5.673, time/batch = 1.267
86520/183960, translation error = 8.380, rotation error = 5.682, time/batch = 1.273
86940/183960, translation error = 8.359, rotation error = 5.670, time/batch = 1.268
87360/183960, translation error = 8.374, rotation error = 5.693, time/batch 

123480/183960, translation error = 9.499, rotation error = 6.161, time/batch = 1.274
123900/183960, translation error = 9.473, rotation error = 6.168, time/batch = 1.270
124320/183960, translation error = 9.466, rotation error = 6.169, time/batch = 1.273
124740/183960, translation error = 9.441, rotation error = 6.165, time/batch = 1.266
125160/183960, translation error = 9.415, rotation error = 6.154, time/batch = 1.284
125580/183960, translation error = 9.390, rotation error = 6.159, time/batch = 1.269
126000/183960, translation error = 9.366, rotation error = 6.155, time/batch = 1.267
126420/183960, translation error = 9.366, rotation error = 6.162, time/batch = 1.271
126840/183960, translation error = 9.353, rotation error = 6.168, time/batch = 1.272
127260/183960, translation error = 9.327, rotation error = 6.158, time/batch = 1.270
127680/183960, translation error = 9.304, rotation error = 6.152, time/batch = 1.272
128100/183960, translation error = 9.307, rotation error = 6.155,

163800/183960, translation error = 11.043, rotation error = 7.076, time/batch = 1.267
164220/183960, translation error = 11.028, rotation error = 7.069, time/batch = 1.272
164640/183960, translation error = 11.014, rotation error = 7.068, time/batch = 1.278
165060/183960, translation error = 10.999, rotation error = 7.063, time/batch = 1.269
165480/183960, translation error = 11.000, rotation error = 7.077, time/batch = 1.273
165900/183960, translation error = 10.984, rotation error = 7.087, time/batch = 1.267
166320/183960, translation error = 10.970, rotation error = 7.088, time/batch = 1.271
166740/183960, translation error = 10.963, rotation error = 7.077, time/batch = 1.277
167160/183960, translation error = 10.956, rotation error = 7.073, time/batch = 1.275
167580/183960, translation error = 10.941, rotation error = 7.082, time/batch = 1.273
168000/183960, translation error = 10.937, rotation error = 7.083, time/batch = 1.274
168420/183960, translation error = 10.950, rotation er

In [9]:
sio.savemat('results.mat', {'trans_pred': np.array(trans_preds), 'trans_gt': np.array(trans_gts), 'uncertainty': np.array(pred_uncertainties)})

if len(pose_map):
    np.savetxt(os.path.join(args.map_dataset, 'map.txt'), np.asarray(pose_map, dtype=np.float32))
    print("map is saved!")

plt.hist(trans_errors, bins='auto')
plt.title("Translation errors")
plt.xlabel("translational error in meters")
plt.ylabel("number of frames")
plt.savefig('terror.png', bbox_inches='tight')

plt.hist(rot_errors, bins='auto')
plt.title("Rotation errors")
plt.xlabel("rotational error in degree")
plt.ylabel("number of frames")
plt.savefig('rerror.png', bbox_inches='tight')

median_trans_errors = np.median(trans_errors)
median_rot_errors = np.median(rot_errors)
mean_trans_errors = np.mean(trans_errors)
mean_rot_errors = np.mean(rot_errors)

print("median translation error = {:.3f}".format(median_trans_errors))
print("median rotation error = {:.3f}".format(median_rot_errors))
print("mean translation error = {:.3f}".format(mean_trans_errors))
print("mean rotation error = {:.3f}".format(mean_rot_errors))   

median translation error = 2.115
median rotation error = 3.713
mean translation error = 10.933
mean rotation error = 7.288


In [10]:
def evaluate(trans_errors,rot_errors):
    t = dataset.last_indexes
    trans_errors_month = list()
    trans_errors_month.append(trans_errors[:t[0]])
    trans_errors_month.append(trans_errors[t[0]:t[1]])
    trans_errors_month.append(trans_errors[t[1]:t[2]])
    trans_errors_month.append(trans_errors[t[2]:t[3]])
    trans_errors_month.append(trans_errors[t[3]:t[4]])
    trans_errors_month.append(trans_errors[t[4]:t[5]])
    trans_errors_month.append(trans_errors[t[5]:t[6]])
    trans_errors_month.append(trans_errors[t[6]:])

    rot_errors_month = list()
    rot_errors_month.append(rot_errors[:t[0]])
    rot_errors_month.append(rot_errors[t[0]:t[1]])
    rot_errors_month.append(rot_errors[t[1]:t[2]])
    rot_errors_month.append(rot_errors[t[2]:t[3]])
    rot_errors_month.append(rot_errors[t[3]:t[4]])
    rot_errors_month.append(rot_errors[t[4]:t[5]])
    rot_errors_month.append(rot_errors[t[5]:t[6]])
    rot_errors_month.append(rot_errors[t[6]:])
    
    print('================== median translation error ==================')
    for trans_errors_i in trans_errors_month:
        print("median translation error = {:.3f}".format(np.median(trans_errors_i)))
        
    print('================== median rotation error ==================')
    for rot_errors_i in rot_errors_month:
        print("median rotation error = {:.3f}".format(np.median(rot_errors_i)))
    
    print('================== mean translation error ==================')
    for trans_errors_i in trans_errors_month:
        print("mean translation error = {:.3f}".format(np.mean(trans_errors_i)))
        
    print('================== mean rotation error ==================')  
    for rot_errors_i in rot_errors_month:
        print("mean rotation error = {:.3f}".format(np.mean(rot_errors_i)))
        
evaluate(trans_errors,rot_errors)

================== median translation error ==================
median translation error = 1.623
median translation error = 1.734
median translation error = 1.973
median translation error = 1.962
median translation error = 2.069
median translation error = 2.195
median translation error = 3.498
median translation error = 2.980
================== median rotation error ==================
median rotation error = 3.076
median rotation error = 3.351
median rotation error = 3.370
median rotation error = 3.444
median rotation error = 3.903
median rotation error = 3.708
median rotation error = 5.170
median rotation error = 4.946
================== mean translation error ==================
mean translation error = 4.109
mean translation error = 4.471
mean translation error = 12.672
mean translation error = 11.863
mean translation error = 10.075
mean translation error = 12.605
mean translation error = 20.390
mean translation error = 13.388
================== mean rotation error ==================
